In [39]:
driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()
driver.get(page_link_to_open)
sleep(1)
homeCard = driver.find_elements(By.CSS_SELECTOR , "a.slider-item")
[c.get_attribute('href') for c in homeCard]

[]

In [26]:
import pandas as pd

In [27]:
from selenium import webdriver  # install selenium
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from time import sleep

In [28]:
path_to_browser = "./chromedriver.exe"
ser = Service(path_to_browser)
op = webdriver.ChromeOptions()
op.add_argument('headless')
page_link_to_open = "https://www.redfin.com/zipcode/00601"

In [111]:
# https://www.redfin.com/CA/San-Francisco/401-Harrison-St-94105/unit-46B/home/144064341
page_link_to_open2 = 'https://www.redfin.com/CA/San-Francisco/401-Harrison-St-94105/unit-46B/home/144064341'

driver = webdriver.Chrome(service=ser, options=op)
driver.maximize_window()
driver.get(page_link_to_open2)
sleep(0.2)
data = driver.find_elements(By.CSS_SELECTOR , ".statsValue")
[price, beds, bath, area] = [c.text for c in data]
stress_address = driver.find_elements(By.CSS_SELECTOR , ".street-address")[0].text
city_and_state = driver.find_elements(By.CSS_SELECTOR , ".dp-subtext")[0].text

# home_facts = driver.find_elements(By.CSS_SELECTOR, ".keyDetailsList")[0] \
#                    .find_elements(By.CSS_SELECTOR, "div.keyDetail")
# home_fact_index = [fact.find_elements(By.CSS_SELECTOR , "span")[0].text for fact in home_facts]
# home_fact_text = [fact.find_elements(By.CSS_SELECTOR, "span.text-right")[0].text for fact in home_facts]
# year_built = home_fact_text[home_fact_index.index('Year Built')] if 'Year Built' in home_fact_index else None
# lot_size = home_fact_text[home_fact_index.index('Lot Size')] if 'Lot Size' in home_fact_index else None

basic_infor = driver.find_elements(By.ID, "basicInfo")[0]
basic_infor_label = [e.text for e in basic_infor.find_elements(By.CSS_SELECTOR, "span.table-label")]
basic_infor_value = [e.text for e in basic_infor.find_elements(By.CSS_SELECTOR, "div.table-value")]
year_built = basic_infor_value[basic_infor_label.index('Year Built')] if 'Year Built' in basic_infor_label else None
county = basic_infor_value[basic_infor_label.index('County')] if 'County' in basic_infor_label else None
# lot_size = basic_infor_value[basic_infor_label.index('Lot Size')] if 'Lot Size' in basic_infor_label else None

scores = driver.find_elements(By.CSS_SELECTOR , ".walk-score")
if scores:
    scores = scores[0].find_elements(By.CSS_SELECTOR , ".score")
    trademark = [score.find_elements(By.CSS_SELECTOR , ".walkscore-trademark")[0].text[:-1] for score in scores]
    score_value = [score.find_elements(By.CSS_SELECTOR , "div.percentage > span.value")[0].text for score in scores]
    walk_score = score_value[trademark.index('Walk Score')] if 'Walk Score' in trademark else None
    transit_score = score_value[trademark.index('Transit Score')] if 'Transit Score' in trademark else None
    bike_score = score_value[trademark.index('Bike Score')] if 'Bike Score' in trademark else None

property_details_container = driver.find_elements(By.CSS_SELECTOR, "div.super-group-content > div.amenity-group")
heating_cooling_text = None
pool_text = None
parking_garage_text = None
lot_text = None

for pc in property_details_container:
    header = pc.find_elements(By.TAG_NAME, "h3")[0].text
    if header == 'Pool Information':
        pool_text = [e.text for e in pc.find_elements(By.TAG_NAME, "li")]
    if header == 'Heating & Cooling':
        heating_cooling_text = [e.text for e in pc.find_elements(By.TAG_NAME, "li")]
    if header == 'Parking & Garage Information':
        parking_garage_text = [e.text for e in pc.find_elements(By.TAG_NAME, "li")]
    if header == 'Lot Information':
        lot_text = [e.text for e in pc.find_elements(By.TAG_NAME, "li")]

photo_elements = driver.find_elements(By.CSS_SELECTOR, 'div.InlinePhotoPreview')
photo_elements = photo_elements[0] if photo_elements else []
img_href = " ".join([img.get_attribute('src') for img in photo_elements.find_elements(By.TAG_NAME, "img")])

In [112]:
img_href

'https://ssl.cdn-redfin.com/system_files/media/622779_JPG/item_10.jpg https://ssl.cdn-redfin.com/system_files/media/622779_JPG/genLdpUgcMediaBrowserUrl/item_29.jpg https://ssl.cdn-redfin.com/system_files/media/622779_JPG/genLdpUgcMediaBrowserUrl/item_85.jpg'

In [71]:
price, beds, bath, area, stress_address[:-1], city_and_state, year_built, lot_size, county, walk_score, transit_score, bike_score

('$7,900,000',
 '3',
 '3',
 '3,214',
 '401 Harrison St Unit 46B',
 'San Francisco, CA 94105',
 '2014',
 '—',
 'San Francisco County',
 '93',
 '100',
 '57')

In [84]:
for lt in lot_text:
    if lt.startswith('Lot Size Acres: '):
        lot_size = lt.split('Lot Size Acres: ')[1]

In [92]:
for hc in heating_cooling_text:
    if hc.startswith('Cooling: '):
        cooling = hc.split('Cooling: ')[1]
    if hc.startswith('Heating: '):
        heating = hc.split('Heating: ')[1]

In [98]:
for p in pool_text:
    if p.startswith('Has Private Pool: '):
        pooling = p.split('Has Private Pool: ')[1]

In [101]:
for pg in parking_garage_text:
    if pg.startswith('Parking Total: '):
        parking_size = pg.split('Parking Total: ')[1]

In [102]:
parking_size

'2'

In [2]:
import pandas as pd
href = pd.read_csv('data\property_href.csv')
href.head()

,zipcode,href
0,1001,https://www.redfin.com/MA/Agawam/24-Rowley-St...
1,1001,https://www.redfin.com/MA/Agawam/418-Meadow-S...
2,1001,https://www.redfin.com/MA/Agawam/611-Suffield...
3,1001,https://www.redfin.com/MA/Agawam/418-Meadow-S...
4,1001,https://www.redfin.com/MA/Agawam/21-Edgewater...


In [3]:
len(href['href'])

568342

In [5]:
import random 
def partition (list_in, n):
    random.shuffle(list_in)
    return [list_in[i::n] for i in range(n)]

set_href = list(set(href['href']))
print(len(set_href))
lists = partition(set_href, 10)

438483


In [17]:
df.to_csv('test.csv', index=False)

In [15]:
df['href'][1]

'https://www.redfin.com/VA/Goochland/Reed-Marsh/Siena-Reed-Marsh/home/175314577'